In [1]:
import os
import h5py
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

# --------------------------------------------------------------------
# 1. Load Training Data (spots/Train) with Slice Information
# --------------------------------------------------------------------
h5_file_path = "/kaggle/input/el-hackathon-2025/elucidata_ai_challenge_data.h5"

with h5py.File(h5_file_path, "r") as f:
    train_spots = f["spots/Train"]
    # Load each slide and tag with its slice name.
    train_spot_tables = {
        slide: pd.DataFrame(np.array(train_spots[slide])).assign(slice_name=slide)
        for slide in train_spots.keys()
    }
# Concatenate all slides into one DataFrame.
train_df = pd.concat(train_spot_tables.values(), ignore_index=True)

# Assume first two columns are coordinates, next 35 are cell abundances.
cell_types = [f"C{i+1}" for i in range(35)]
train_df.columns = ["x", "y"] + cell_types + ["slice_name"]
print("Training data shape:", train_df.shape)

# --------------------------------------------------------------------
# 2. Compute Ranks of Cell Abundances for Each Spot
# --------------------------------------------------------------------
# Here we compute descending ranks: highest abundance gets rank 1.
# The DataFrame.rank method is applied row-wise (axis=1).
ranks = train_df[cell_types].rank(axis=1, method="dense", ascending=False).values

# --------------------------------------------------------------------
# 3. Define the Foundation Autoencoder (learns embeddings from cell abundance ranks)
# --------------------------------------------------------------------
class FoundationAutoencoder(nn.Module):
    def __init__(self, input_dim=35, embed_dim=16, hidden_dim=64):
        super(FoundationAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, embed_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )
    def forward(self, x):
        emb = self.encoder(x)
        recon = self.decoder(emb)
        return emb, recon

# Dataset for foundation autoencoder training on ranks.
class FoundationDataset(Dataset):
    def __init__(self, rank_values):
        self.data = torch.tensor(rank_values, dtype=torch.float32)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

def train_foundation_autoencoder(model, dataloader, num_epochs=20, lr=0.001, device='cpu'):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for data in dataloader:
            data = data.to(device)
            emb, recon = model(data)
            loss = criterion(recon, data)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * data.size(0)
        print(f"Foundation Autoencoder Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(dataloader.dataset):.4f}")
    return model

# Create dataset and dataloader using the computed ranks.
foundation_dataset = FoundationDataset(ranks)
foundation_loader = DataLoader(foundation_dataset, batch_size=32, shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
foundation_model = FoundationAutoencoder(input_dim=35, embed_dim=16, hidden_dim=64)
print("Training Foundation Autoencoder on cell abundance ranks...")
foundation_model = train_foundation_autoencoder(foundation_model, foundation_loader, num_epochs=30, lr=0.001, device=device)


Training data shape: (8349, 38)
Training Foundation Autoencoder on cell abundance ranks...
Foundation Autoencoder Epoch 1/30, Loss: 73.2596
Foundation Autoencoder Epoch 2/30, Loss: 21.7166
Foundation Autoencoder Epoch 3/30, Loss: 17.0590
Foundation Autoencoder Epoch 4/30, Loss: 14.7168
Foundation Autoencoder Epoch 5/30, Loss: 12.5333
Foundation Autoencoder Epoch 6/30, Loss: 10.5480
Foundation Autoencoder Epoch 7/30, Loss: 9.3224
Foundation Autoencoder Epoch 8/30, Loss: 8.6597
Foundation Autoencoder Epoch 9/30, Loss: 8.1054
Foundation Autoencoder Epoch 10/30, Loss: 7.6761
Foundation Autoencoder Epoch 11/30, Loss: 7.3222
Foundation Autoencoder Epoch 12/30, Loss: 7.0932
Foundation Autoencoder Epoch 13/30, Loss: 6.8945
Foundation Autoencoder Epoch 14/30, Loss: 6.7042
Foundation Autoencoder Epoch 15/30, Loss: 6.5794
Foundation Autoencoder Epoch 16/30, Loss: 6.4914
Foundation Autoencoder Epoch 17/30, Loss: 6.4058
Foundation Autoencoder Epoch 18/30, Loss: 6.3701
Foundation Autoencoder Epoch 1

In [2]:

# --------------------------------------------------------------------
# 3. Precompute Foundation Embeddings for All Training Spots
# --------------------------------------------------------------------
with torch.no_grad():
    foundation_model.eval()
    train_abundances = torch.tensor(train_df[cell_types].values, dtype=torch.float32).to(device)
    train_embeddings, _ = foundation_model(train_abundances)
    train_embeddings = train_embeddings.cpu().numpy()

# --------------------------------------------------------------------
# 4. Define Dataset Classes that Load Images from the H5 File
# --------------------------------------------------------------------
class MainMappingWithImageH5Dataset(Dataset):
    """
    For training: Maps (x, y) coordinates and the corresponding image patch 
    (loaded from the H5 file) to the precomputed foundation embedding.
    Expects a DataFrame with columns: "x", "y", and "slice_name".
    """
    def __init__(self, df, embeddings, h5_file_path, patch_size=64, transform=None, train=True):
        self.df = df.reset_index(drop=True)
        self.embeddings = embeddings  # Precomputed embeddings in the same order as df.
        self.patch_size = patch_size
        self.transform = transform if transform is not None else transforms.ToTensor()
        self.h5_file_path = h5_file_path
        self.train = train
        self.images = {}
        group = "Train" if train else "Test"
        # Load images for each unique slice from the H5 file.
        with h5py.File(self.h5_file_path, "r") as f:
            for slice_name in self.df['slice_name'].unique():
                img_array = np.array(f[f"images/{group}"][slice_name])
                # Normalize and convert to uint8 if necessary.
                if img_array.dtype != np.uint8:
                    img_array = img_array - img_array.min()
                    if img_array.max() > 0:
                        img_array = img_array / img_array.max()
                    img_array = (img_array * 255).astype(np.uint8)
                if img_array.ndim > 3:
                    img_array = np.squeeze(img_array)
                # If the image is grayscale (2D), convert to RGB.
                if img_array.ndim == 2:
                    img_array = np.stack([img_array]*3, axis=-1)
                if img_array.shape[-1] != 3:
                    raise ValueError(f"Unexpected number of channels in image for slice {slice_name}: {img_array.shape}")
                self.images[slice_name] = Image.fromarray(img_array, mode="RGB")
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        coord = np.array([row['x'], row['y']], dtype=np.float32)
        slice_name = row['slice_name']
        image = self.images[slice_name]
        x, y = int(row['x']), int(row['y'])
        half_patch = self.patch_size // 2
        left = max(x - half_patch, 0)
        upper = max(y - half_patch, 0)
        right = left + self.patch_size
        lower = upper + self.patch_size
        patch = image.crop((left, upper, right, lower))
        patch = self.transform(patch)
        target = self.embeddings[idx]
        return torch.tensor(coord, dtype=torch.float32), patch, torch.tensor(target, dtype=torch.float32)

class TestMappingWithImageH5Dataset(Dataset):
    """
    For testing: Maps (x, y) coordinates and the corresponding image patch 
    (loaded from the H5 file) to be used for prediction.
    Expects a DataFrame with columns: "x", "y", and "slice_name".
    """
    def __init__(self, df, h5_file_path, patch_size=64, transform=None):
        self.df = df.reset_index(drop=True)
        self.patch_size = patch_size
        self.transform = transform if transform is not None else transforms.ToTensor()
        self.h5_file_path = h5_file_path
        self.images = {}
        group = "Test"
        with h5py.File(self.h5_file_path, "r") as f:
            for slice_name in self.df['slice_name'].unique():
                img_array = np.array(f[f"images/{group}"][slice_name])
                if img_array.dtype != np.uint8:
                    img_array = img_array - img_array.min()
                    if img_array.max() > 0:
                        img_array = img_array / img_array.max()
                    img_array = (img_array * 255).astype(np.uint8)
                if img_array.ndim > 3:
                    img_array = np.squeeze(img_array)
                if img_array.ndim == 2:
                    img_array = np.stack([img_array]*3, axis=-1)
                if img_array.shape[-1] != 3:
                    raise ValueError(f"Unexpected number of channels in image for slice {slice_name}: {img_array.shape}")
                self.images[slice_name] = Image.fromarray(img_array, mode="RGB")
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        coord = np.array([row['x'], row['y']], dtype=np.float32)
        slice_name = row['slice_name']
        image = self.images[slice_name]
        x, y = int(row['x']), int(row['y'])
        half_patch = self.patch_size // 2
        left = max(x - half_patch, 0)
        upper = max(y - half_patch, 0)
        right = left + self.patch_size
        lower = upper + self.patch_size
        patch = image.crop((left, upper, right, lower))
        patch = self.transform(patch)
        return torch.tensor(coord, dtype=torch.float32), patch

# --------------------------------------------------------------------
# 5. Define the Main Model that Uses Image Patches and Coordinates
# --------------------------------------------------------------------
class MainModelMappingWithImage(nn.Module):
    def __init__(self, coord_input_dim=2, patch_channels=3, patch_size=64, embed_dim=16, hidden_dim=64):
        super(MainModelMappingWithImage, self).__init__()
        # CNN to encode image patches.
        self.image_encoder = nn.Sequential(
            nn.Conv2d(patch_channels, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten()
        )
        # Calculate the image feature dimension.
        img_feat_dim = 32 * (patch_size // 4) * (patch_size // 4)
        # Fully connected layers to map concatenated [coords, image_features] to embedding.
        self.fc = nn.Sequential(
            nn.Linear(coord_input_dim + img_feat_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, embed_dim)
        )
    
    def forward(self, coords, image_patches):
        img_features = self.image_encoder(image_patches)
        combined = torch.cat([coords, img_features], dim=1)
        embedding = self.fc(combined)
        return embedding

def train_main_mapping_with_image(model, dataloader, num_epochs=20, lr=0.001, device='cpu'):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for coords, patches, target_emb in dataloader:
            coords = coords.to(device)
            patches = patches.to(device)
            target_emb = target_emb.to(device)
            pred_emb = model(coords, patches)
            loss = criterion(pred_emb, target_emb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * coords.size(0)
        print(f"Main Mapping With Image Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(dataloader.dataset):.4f}")
    return model

# --------------------------------------------------------------------
# 6. Create Dataset and Train the Main Mapping with Image Model (Training)
# --------------------------------------------------------------------
patch_size = 64

# Create the training dataset using images from the H5 file.
main_image_dataset = MainMappingWithImageH5Dataset(
    train_df, train_embeddings, h5_file_path, patch_size=patch_size, transform=transforms.ToTensor(), train=True
)
main_image_loader = DataLoader(main_image_dataset, batch_size=32, shuffle=True)

main_model_img = MainModelMappingWithImage(coord_input_dim=2, patch_channels=3, patch_size=patch_size, embed_dim=16, hidden_dim=64)
print("Training Main Mapping With Image Model...")
main_model_img = train_main_mapping_with_image(main_model_img, main_image_loader, num_epochs=200, lr=0.001, device=device)

# --------------------------------------------------------------------
# 7. Inference on Test Data and Submission Creation
# --------------------------------------------------------------------
# Load test spots for slide "S_7" from the H5 file.
with h5py.File(h5_file_path, "r") as f:
    test_spots = f["spots/Test"]
    test_array = np.array(test_spots["S_7"])
    test_df = pd.DataFrame(test_array)
# Test file has three columns: x, y, Test_set. Drop the third column.
if test_df.shape[1] == 3:
    test_df.columns = ["x", "y", "Test_set"]
    test_df = test_df[["x", "y"]]
# Add slice_name column so we know which image to load.
test_df["slice_name"] = "S_7"
print("Test data shape:", test_df.shape)

# Create the test dataset (loading images from H5).
test_dataset = TestMappingWithImageH5Dataset(test_df, h5_file_path, patch_size=patch_size, transform=transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Predict foundation embeddings from test spots using the main mapping model with image.
predicted_embeddings_list = []
main_model_img.eval()
with torch.no_grad():
    for coords, patches in test_loader:
        coords = coords.to(device)
        patches = patches.to(device)
        pred_emb = main_model_img(coords, patches)
        predicted_embeddings_list.append(pred_emb)
    predicted_embeddings = torch.cat(predicted_embeddings_list, dim=0)

# Use the foundation decoder to convert predicted embeddings into cell abundance predictions.
foundation_model.eval()
with torch.no_grad():
    predicted_abundances = foundation_model.decoder(predicted_embeddings)
    predicted_abundances = predicted_abundances.cpu().numpy()

# Create submission DataFrame and save CSV.
submission_df = pd.DataFrame(predicted_abundances, columns=cell_types)
submission_df.insert(0, 'ID', test_df.index)
submission_file = "submission.csv"
submission_df.to_csv(submission_file, index=False)
print(f"Submission file '{submission_file}' created!")


Training Main Mapping With Image Model...
Main Mapping With Image Epoch 1/200, Loss: 1.2825
Main Mapping With Image Epoch 2/200, Loss: 1.1551
Main Mapping With Image Epoch 3/200, Loss: 1.0690
Main Mapping With Image Epoch 4/200, Loss: 0.9969
Main Mapping With Image Epoch 5/200, Loss: 0.9599
Main Mapping With Image Epoch 6/200, Loss: 0.9170
Main Mapping With Image Epoch 7/200, Loss: 0.8937
Main Mapping With Image Epoch 8/200, Loss: 0.8848
Main Mapping With Image Epoch 9/200, Loss: 0.8193
Main Mapping With Image Epoch 10/200, Loss: 0.7880
Main Mapping With Image Epoch 11/200, Loss: 0.7780
Main Mapping With Image Epoch 12/200, Loss: 0.7522
Main Mapping With Image Epoch 13/200, Loss: 0.7302
Main Mapping With Image Epoch 14/200, Loss: 0.6984
Main Mapping With Image Epoch 15/200, Loss: 0.6792
Main Mapping With Image Epoch 16/200, Loss: 0.6711
Main Mapping With Image Epoch 17/200, Loss: 0.6261
Main Mapping With Image Epoch 18/200, Loss: 0.6251
Main Mapping With Image Epoch 19/200, Loss: 0.600

In [3]:
submission_df

,ID,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35
0,0,0.572766,0.287734,0.381983,0.410640,0.715955,0.292551,0.494569,-0.011110,0.335466,...,0.772406,0.121624,0.213885,0.091956,0.077566,0.527763,0.264700,0.613886,-0.074841,0.427175
1,1,0.552722,0.300106,0.374616,0.500414,1.365019,0.321056,0.568109,0.001917,0.411381,...,0.755617,0.206278,0.221838,0.309310,0.285798,0.567685,0.298599,0.723431,0.017861,0.422775
2,2,0.900694,0.455989,0.629256,1.167453,1.098811,1.003285,1.026470,0.240135,1.771432,...,1.255766,1.139969,0.667567,0.450991,0.677158,0.895535,0.649095,0.847810,-0.077062,0.727964
3,3,0.604456,0.372985,0.468118,0.846613,2.923739,0.426083,0.764588,0.058285,0.611131,...,0.821667,0.414098,0.414424,0.768732,0.501714,0.769868,0.439355,1.083667,0.277532,0.485925
4,4,0.572766,0.287734,0.381983,0.410640,0.715955,0.292551,0.494569,-0.011110,0.335466,...,0.772406,0.121624,0.213885,0.091956,0.077566,0.527763,0.264700,0.613886,-0.074841,0.427175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,2083,0.630716,0.304699,0.412211,0.523903,0.801066,0.447446,0.600817,0.037918,0.617038,...,0.757201,0.338572,0.217217,0.058940,0.036302,0.571248,0.324799,0.710127,-0.033185,0.466247
2084,2084,0.571617,0.289127,0.382255,0.412300,0.719323,0.294402,0.498913,-0.014990,0.345820,...,0.774242,0.126915,0.214026,0.090772,0.076137,0.528586,0.266974,0.615131,-0.078409,0.429954
2085,2085,0.626791,0.340694,0.434470,0.641740,0.904438,0.489417,0.692761,0.067711,0.837497,...,0.904676,0.547586,0.301920,0.177181,0.316479,0.621201,0.400350,0.529042,-0.195772,0.546351
2086,2086,1.148310,0.589998,0.893282,1.818598,1.561878,1.296989,1.308555,0.440859,2.489008,...,1.774328,1.865028,1.317348,0.837126,1.294322,1.224849,0.956547,0.811471,-0.281313,1.098412
